In [1]:
import os

# Set the Spark version to 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [556 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,274 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,398 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,009 kB]
Get

In [2]:
# Import necessary libraries
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()


In [3]:
# Read in the AWS S3 bucket into a DataFrame
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/partitioned_home_sales_view_revised.csv"
spark.sparkContext.addFile(url)
partitioned_home_sales_view_df = spark.read.csv(SparkFiles.get("partitioned_home_sales_view_revised.csv"), sep=",", header=True)
partitioned_home_sales_view_df.show()


# Creating a temporary table from the DataFrame
df_home_sales.createOrReplaceTempView('home_sales')

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# Create a temporary view of the DataFrame
partitioned_home_sales_view_df.createOrReplaceTempView('partitioned_home_sales_view')


In [5]:
# SQL queries

# 1. Average price for a four-bedroom house sold in each year
sql_1 = """
SELECT
  YEAR(date) AS YEAR,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM partitioned_home_sales_view
WHERE bedrooms = 4
GROUP BY YEAR
ORDER BY YEAR DESC
"""
spark.sql(sql_1).show()



+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    296363.88|
|2021|    301819.44|
|2020|    298353.78|
|2019|     300263.7|
+----+-------------+



In [6]:
# 2. Average price of a home for each year built with 3 bedrooms and 3 bathrooms
sql_2 = """
SELECT
  YEAR(date_built) AS YEAR,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM partitioned_home_sales_view
WHERE bedrooms = 3
AND bathrooms = 3
GROUP BY YEAR(date_built)
ORDER BY YEAR DESC
"""
spark.sql(sql_2).show()


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2017|    292676.79|
|2016|    290555.07|
|2015|     288770.3|
|2014|    290852.27|
|2013|    295962.27|
|2012|    293683.19|
|2011|    291117.47|
|2010|    292859.62|
+----+-------------+



In [7]:
# 3. Average price of a home for each year built with specific criteria
sql_3 = """
SELECT
  YEAR(date_built) AS YEAR_BUILT,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM partitioned_home_sales_view
WHERE bedrooms = 3
AND bathrooms = 3
AND sqft_living >= 2000
AND floors = 2
GROUP BY YEAR_BUILT
ORDER BY YEAR_BUILT DESC
"""
spark.sql(sql_3).show()


+----------+-------------+
|YEAR_BUILT|AVERAGE_PRICE|
+----------+-------------+
|      2017|    280317.58|
|      2016|     293965.1|
|      2015|    297609.97|
|      2014|    298264.72|
|      2013|    303676.79|
|      2012|    307539.97|
|      2011|    276553.81|
|      2010|    285010.22|
+----------+-------------+



In [8]:
# Timing the query for average prices of homes with views and prices above a threshold
start_query_time = time.time()

query_view_rating_avg_price = """
SELECT
  view AS home_view,
  ROUND(AVG(price), 2) AS avg_price
FROM partitioned_home_sales_view
WHERE price >= 350000
GROUP BY home_view
ORDER BY home_view DESC
"""
result_view_rating_avg_price = spark.sql(query_view_rating_avg_price)
result_view_rating_avg_price.show()


elapsed_time_query = time.time() - start_query_time
print(f"Query elapsed time: {elapsed_time_query:.2f} seconds")


+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 1.2471935749053955 seconds ---


In [9]:
# Caching the partitioned_home_sales_view data table
spark.sql('CACHE TABLE partitioned_home_sales_view')


DataFrame[]

In [10]:
# Checking cache status of the partitioned_home_sales_view table
partitioned_home_sales_view_cache_status = spark.catalog.isCached('partitioned_home_sales_view')
print(f"Home sales table cached: {partitioned_home_sales_view_cache_status}")


True

In [11]:
# Re-running the query with cached data
start_cached_query_time = time.time()

result_cached_query = spark.sql(query_view_rating_avg_price)
result_cached_query.show()

elapsed_time_cached_query = time.time() - start_cached_query_time
print(f"Cached query elapsed time: {elapsed_time_cached_query:.2f} seconds")


+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 1.0824172496795654 seconds ---


In [12]:
# Writing the DataFrame to a Parquet file, partitioned by date built
parquet_output_path = '/tmp/partitioned_partitioned_home_sales_view'
partitioned_home_sales_view_df.write.partitionBy('date_built').parquet(parquet_output_path, mode='overwrite')


In [13]:
# Reading from the partitioned Parquet file
partitioned_partitioned_home_sales_view_data = spark.read.parquet(parquet_output_path)

# Creating a temporary table from the DataFrame
df_home_sales.createOrReplaceTempView('home_sales')

In [14]:
# Creating a new temporary view for the partitioned data
partitioned_partitioned_home_sales_view_data.createOrReplaceTempView('partitioned_partitioned_home_sales_view')


In [15]:
# Running a query on the Parquet DataFrame and timing it
time_at_query_start_parquet = time.time()

query_view_rating_parquet = """
SELECT
  view AS view_category,
  ROUND(AVG(price), 2) AS average_home_price
FROM partitioned_partitioned_home_sales_view
WHERE price >= 350000
GROUP BY view_category
ORDER BY view_category DESC
"""
result_view_rating_parquet = spark.sql(query_view_rating_parquet)
result_view_rating_parquet.show()

elapsed_time_parquet_query = time.time() - time_at_query_start_parquet
print(f"--- Parquet query runtime: {elapsed_time_parquet_query:.2f} seconds ---")


+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 1.384225606918335 seconds ---


In [16]:
# Uncaching the partitioned_home_sales_view table
spark.sql('UNCACHE TABLE partitioned_home_sales_view')


DataFrame[]

In [17]:
# Verifying if the partitioned_home_sales_view table has been uncached
cache_status_partitioned_home_sales_view = spark.catalog.isCached('partitioned_home_sales_view')
if cache_status_partitioned_home_sales_view:
    print('partitioned_home_sales_view is still cached')
else:
    print('all clear')

all clear


In [ ]:

SELECT year(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year_built


In [ ]:

SELECT year(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND square_footage >= 2000
GROUP BY year_built


In [ ]:

# Partitioning the DataFrame by 'date_built' and saving as Parquet
df_home_sales.write.partitionBy('date_built').parquet('home_sales_parquet')

# Reading the partitioned Parquet data
df_home_sales_parquet = spark.read.parquet('home_sales_parquet')

# Creating a temporary table from the Parquet DataFrame
df_home_sales_parquet.createOrReplaceTempView('home_sales_parquet')


In [ ]:

SELECT year(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year_built


In [ ]:

SELECT year(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND square_footage >= 2000
GROUP BY year_built


In [ ]:

# Checking if the table is cached
cache_status = spark.catalog.isCached('home_sales')
print('Is home_sales cached:', cache_status)

# Running query on cached table (example)
# ...

# Partitioning by 'date_built' and handling parquet data
# ...

# Creating temporary table for parquet data
# ...

# Running query on parquet table (example)
# ...

# Uncaching the home_sales temporary table
spark.sql('UNCACHE TABLE home_sales')

# Verifying uncache
uncache_status = not spark.catalog.isCached('home_sales')
print('Is home_sales uncached:', uncache_status)


In [ ]:

SELECT year(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year_built


In [ ]:

SELECT year(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND square_footage >= 2000
GROUP BY year_built


In [ ]:

SELECT year(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year(date_built)


In [ ]:

SELECT year(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND square_footage >= 2000
GROUP BY year(date_built)


In [ ]:

# Cache the temporary table
spark.sql("CACHE TABLE home_sales")

# Check if table is cached
is_cached = spark.catalog.isCached("home_sales")
print("Is home_sales cached:", is_cached)

# Run query on cached table (Placeholder for actual query)
# ...

# Partitioning by 'date_built' and handling parquet data
# ...

# Creating temporary table for parquet data
# ...

# Running query on parquet table (Placeholder for actual query)
# ...

# Uncaching the home_sales temporary table
spark.sql("UNCACHE TABLE home_sales")

# Verifying uncache
is_uncached = not spark.catalog.isCached("home_sales")
print("Is home_sales uncached:", is_uncached)


In [ ]:

SELECT year(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year(date_built)


In [ ]:

SELECT year(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND square_footage >= 2000
GROUP BY year(date_built)


In [ ]:

# Cache the temporary table
spark.sql("CACHE TABLE home_sales")

# Check if table is cached
is_cached = spark.catalog.isCached("home_sales")
print("Is home_sales cached:", is_cached)

# Partitioning by 'date_built' and handling parquet data
# ...

# Creating temporary table for parquet data
# ...

# Running query on parquet table (Placeholder for actual query)
# ...

# Uncaching the home_sales temporary table
spark.sql("UNCACHE TABLE home_sales")

# Verifying uncache
is_uncached = not spark.catalog.isCached("home_sales")
print("Is home_sales uncached:", is_uncached)


In [ ]:

SELECT year(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year(date_built)


In [ ]:

SELECT year(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND square_footage >= 2000
GROUP BY year(date_built)


In [ ]:

# Cache the temporary table
spark.sql("CACHE TABLE home_sales")

# Check if table is cached
is_cached = spark.catalog.isCached("home_sales")
print("Is home_sales cached:", is_cached)


In [ ]:

# Running query on cached table (Placeholder for actual query)
# ...


In [ ]:

# Partitioning by 'date_built' and handling parquet data (Placeholder)
# ...

# Creating temporary table for parquet data (Placeholder)
# ...


In [ ]:

# Running query on parquet table (Placeholder for actual query)
# ...


In [ ]:

# Uncaching the home_sales temporary table
spark.sql("UNCACHE TABLE home_sales")

# Verifying uncache
is_uncached = not spark.catalog.isCached("home_sales")
print("Is home_sales uncached:", is_uncached)


In [ ]:

SELECT year(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year(date_built)


In [ ]:

SELECT year(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND square_footage >= 2000
GROUP BY year(date_built)


In [ ]:

# Cache, partition, and query on cached and partitioned data (placeholder code)
# ... (implement as per specific requirements and dataset)


In [ ]:

SELECT year(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year(date_built)


In [ ]:

SELECT year(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND square_footage >= 2000
GROUP BY year(date_built)


In [ ]:

# Placeholder for additional operations such as caching, partitioning, etc.
# (Actual implementation depends on specific data and schema details)


In [ ]:

SELECT year(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year(date_built)


In [ ]:

SELECT year(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND square_footage >= 2000
GROUP BY year(date_built)


In [ ]:

# Placeholder for additional operations such as caching, partitioning, etc.
# (Actual implementation depends on specific data and schema details)
